In [1]:
# import mpl_finance
# import matplotlib.dates as mdates 
from talib import MA,STOCH
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
START_OF_2018 = 6044
START_OF_2017 = 5793
TIME_STEP = 30

In [3]:
# big_four = df[["Date", "Open", "High", "Low", "Close"]] 6046 -> 6044 (2018)
# big_four_2018 = big_four[START_OF_2018:] 5795 -> 5793 (2017)
# ax = plt.subplot()
# mpl_finance.candlestick_ohlc(ax, big_four_2018, width=5, colorup='g', colordown='r')
# ax.xaxis_date()
# ax.grid(True)
# plt.show()
# date_2018 = df[START_OF_2018:]["Date"]

In [4]:
df = pd.read_csv("SPY.csv",date_parser=True)

In [5]:
df_train = df[ df["Date"]<'2017-01-01' ].copy()
df_validation = df[(df['Date'] >= '2017-01-01') & (df['Date'] < "2018-01-01")].copy()
df_plot = df[df["Date"] >= "2018-01-01"].copy()
# df_validation.tail()

# Graph Plotting

In [6]:
# ma_10_2018 = MA(df[START_OF_2018 - 10:]["Close"], timeperiod=10, matype=0)
# ma_30_2018 = MA(df[START_OF_2018 - 30:]["Close"], timeperiod=30, matype=0)
# df_2018_neg8 = df[START_OF_2018-8:]
# k_2018, d_2018 = STOCH(df_2018_neg8["High"], df_2018_neg8["Low"], df_2018_neg8["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# plt.plot(np.arange(len(ma_10_2018) - 10), ma_10_2018[10:], label="MA-10")
# plt.plot(np.arange(len(ma_30_2018) - 30), ma_30_2018[30:], label="MA-30")
# plt.legend()
# plt.show()

# Add Technical Analysis

In [7]:
ma_10_train = MA(df_train["Close"], timeperiod=10, matype=0)
ma_30_train = MA(df_train["Close"], timeperiod=30, matype=0)
k_train, d_train = STOCH(df_train["High"], df_train["Low"], df_train["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_train["MA10"] = ma_10_train
df_train["MA30"] = ma_30_train
df_train["K"] = k_train
df_train["D"] = d_train

ma_10_validation = MA(df_validation["Close"], timeperiod=10, matype=0)
ma_30_validation = MA(df_validation["Close"], timeperiod=30, matype=0)
k_validation, d_validation = STOCH(df_validation["High"], df_validation["Low"], df_validation["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_validation["MA10"] = ma_10_validation
df_validation["MA30"] = ma_30_validation
df_validation["K"] = k_validation
df_validation["D"] = d_validation

In [8]:
df_validation.shape

(251, 11)

## Drop dates and NA

In [9]:
df_train = df_train.dropna(axis=0)
df_train = df_train.drop(["Date"], axis=1)

df_validation = df_validation.dropna(axis=0)
df_validation = df_validation.drop(["Date"], axis=1)

In [13]:
df_validation.shape

(222, 10)

## Normalize data

In [15]:
def normalizeDataframe(data_frame):
    normalize_df = data_frame.copy()
    for column in normalize_df.columns:
        min_value = min(normalize_df[column])
        max_value = max(normalize_df[column])
        normalize_df[column] = (normalize_df[column] - min_value) / (max_value - min_value)
    return normalize_df

In [16]:
df_train = normalizeDataframe(df_train)
df_validation = normalizeDataframe(df_validation)

In [19]:
df_validation.shape

(222, 10)

## Prepare X, y train and validation for RNN

In [20]:
data_train = df_train.values
data_validation = df_validation.values

In [21]:
X_train = []
y_train = []
X_validation = []
y_validation = []

In [22]:
for i in range(30,data_train.shape[0]):
    X_train.append(data_train[i-30:i])
    y_train.append(data_train[i, 0])

for i in range(30, data_validation.shape[0]):
    X_validation.append(data_validation[i-30:i])
    y_validation.append(data_validation[i,0])

In [23]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_validation, y_validation = np.array(X_validation), np.array(y_validation)

In [31]:
X_train.shape
y_train.shape

(5734,)

# Building LSTM

In [26]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [27]:
regressior = Sequential()

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 120, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 1))

In [28]:
regressior.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 60)            17040     
_________________________________________________________________
dropout (Dropout)            (None, 30, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 60)            29040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 60)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 80)            45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 80)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               9

In [29]:
regressior.compile(optimizer='adam', loss = 'mean_squared_error')

In [30]:
regressior.fit(X_train, y_train, epochs=64, batch_size=60, validation_data = (X_validation, y_validation))

Train on 5734 samples, validate on 192 samples
Epoch 1/64
5734/5734 [==============================] - 18s 3ms/sample - loss: 0.0198 - val_loss: 0.0090
Epoch 2/64
5734/5734 [==============================] - 11s 2ms/sample - loss: 0.0027 - val_loss: 0.0068
Epoch 3/64
5734/5734 [==============================] - 12s 2ms/sample - loss: 0.0027 - val_loss: 0.0064
Epoch 4/64
5734/5734 [==============================] - 12s 2ms/sample - loss: 0.0023 - val_loss: 0.0050
Epoch 5/64
5734/5734 [==============================] - 12s 2ms/sample - loss: 0.0021 - val_loss: 0.0113
Epoch 6/64
5734/5734 [==============================] - 13s 2ms/sample - loss: 0.0019 - val_loss: 0.0101
Epoch 7/64
5734/5734 [==============================] - 14s 2ms/sample - loss: 0.0020 - val_loss: 0.0107
Epoch 8/64
5734/5734 [==============================] - 14s 2ms/sample - loss: 0.0019 - val_loss: 0.0122
Epoch 9/64
5734/5734 [==============================] - 14s 2ms/sample - loss: 0.0018 - val_loss: 0.0085
Epoch 10

In [ ]:
# ma_10_2017 = MA(df[START_OF_2017 - 10 : START_OF_2018]["Close"], timeperiod=10, matype=0)
# ma_30_2017 = MA(df[START_OF_2017 - 30 : START_OF_2018]["Close"], timeperiod=30, matype=0)
# df_2017_neg8 = df[START_OF_2017 - 8 : START_OF_2018]
# k_2017, d_2017 = STOCH(df_2017_neg8["High"], df_2017_neg8["Low"], df_2017_neg8["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

In [24]:
# def removeNanFromNumpyArray(numpy_array):
#     temp = numpy_array.copy()
#     temp = temp[np.logical_not(np.isnan(temp))]
#     return temp
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))

5793
5793
5793
5793


In [25]:
# ma_10_train = removeNanFromNumpyArray(ma_10_train)
# ma_30_train = removeNanFromNumpyArray(ma_30_train)
# k_train = removeNanFromNumpyArray(k_train)
# d_train = removeNanFromNumpyArray(d_train)

In [26]:
# def normalizeNumpyArray(numpy_array):
#     min_val = min(numpy_array)
#     max_val = max(numpy_array)
#     normalize_equation = lambda x: (x - min_val)/(max_val - min_val)
#     normalized_numpy_array = np.array([normalize_equation(x) for x in numpy_array])
#     return normalized_numpy_array
    

In [27]:
# ma_10_train = normalizeNumpyArray(ma_10_train)
# ma_30_train = normalizeNumpyArray(ma_30_train)
# k_train = normalizeNumpyArray(k_train)
# d_train = normalizeNumpyArray(d_train)

In [28]:
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))
# print(df_train.shape)

5784
5764
5785
5785
(5793, 7)


In [29]:
# ma_10_train = ma_10_train[20:]
# k_train = k_train[21:]
# d_train = d_train[21:]

In [30]:
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))

5764
5764
5764
5764
